In [2]:
import sklearn
assert sklearn.__version__ >= "0.20"

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >="2.0"

import numpy as np
import os

np.random.seed(42)
tf.random.set_seed(42)

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation='nearest')
    plt.axis("off")
    
def plot_color_image(image):
    plt.imshow(image, interpolation='nearest')
    plt.axis("off")

# What is a Convolution?

In [ ]:
from sklearn.datasets import load_sample_image

china=load_sample_image("china.jpg")/255
flower=load_sample_image("flower.jpg")/255
images=np.array([china, flower])
batch_size, height, width, channels = images.shape

filters=np.zeros(shape=(7,7,channels, 2),dtype=np.float32)
filters[:,3,:,0] =1 # vertical line
filters[3,:,:,1] =1 # horizontal line

outputs = tf.nn.conv2d(images, filters, strides=1, padding="SAME")

plt.imshow(outputs[0,:,:,1], cmap='gray')
plt.axis('off')

In [ ]:
for image_index in (0,1):
    for feature_map_index in (0,1):
        plt.subplot(2,2, image_index*2 + feature_map_index+1)
        plot_image(outputs[image_index,:,:,feature_map_index])


In [ ]:
def crop(image):
    return images[150:220, 130:250]

In [ ]:
plot_image(crop(images[0,:,:,0]))

for feature_map_index, filename in enumerate(["china_vertical", "china_horizontal"]):
    plot_image(crop(outputs[0,:,:,feature_map_index]))
    

In [ ]:
plot_image(filters[:,:,0,0])
plot_image(filters[:,:,0,1])

## Convolutional Layer

In [ ]:
conv = keras.layers.Conv2D(filters =2, kernel_size=7, strides=1,
                          padding="SAME", activation='relu', input_shape=outputs.shape)

In [ ]:
conv_outputs=conv(images)
conv_outputs.shape

次元はバッチサイズ、高さ、幅、チャンネル。  
この畳込み層は2つのフィルターを持つのでチャンネルの次元数は2になる。

In [ ]:
plt.figure(figsize =(10,6))
for image_index in (0,1):
    for feature_map_index in (0,1):
        plt.subplot(2,2, image_index*2+feature_map_index+1 )
        plot_image(crop(conv_outputs[image_index, :,:,feature_map_index]))


In [ ]:
conv.set_weights([filters, np.zeros(2)])

In [ ]:
conv_outputs=conv(images)
conv_outputs.shape

In [ ]:
plt.figure(figsize =(10,6))
for image_index in (0,1):
    for feature_map_index in (0,1):
        plt.subplot(2,2, image_index*2+feature_map_index+1 )
        plot_image(crop(conv_outputs[image_index, :,:,feature_map_index]))


## VALID vs SAME padding

In [ ]:
def feature_map_size(input_size, kernel_size, strides=1, padding="SAME"):
    if padding=='SAME':
        return (input_size-1)//strides +1
    else:
        return (input_size -kernel_size)//strides+1 

In [ ]:
def pad_before_and_padded_size(input_size, kernel_size, strides=1):
    fmap_size=feature_map_size(input_size, kernel_size, strides)
    padded_size=max((fmap_size -1)*strides + kernel_size, input_size)
    pad_before=(padded_size-input_size)//2
    return pad_before, padded_size

In [ ]:
def manual_same_padding(images, kernel_size, strides=1):
    if kernel_size==1:
        return images.astype(np.float32)
    batch_size, height, width, channels=images.shape
    top_pad, padded_height=pad_before_and_padded_size(height, kernel_size, strides)
    left_pad, padded_widtht=pad_before_and_padded_size(width, kernel_size, strides)
    padded_shape  = [batch_size, padded_height, padded_width, channels]
    padded_images=np.zeros(padded_shape, dtype=np.float32)
    padded_images[:, top_pad:hieght+top_pad, left_pad:width+left_pad, :]=images
    return padded_images

In [ ]:
kernel_size =7
strides =2

conv_valid = keras.layers.Conv2D(filters=1, kernel_size=kernel_size, strides=strides, padding="VALID")
conv_same = keras.layers.Conv2D(filters=1, kernel_size=kernel_size, strides=strides, padding="SAME")

valid_output = conv_valid(manual_same_padding(images, kernel_size, strides))

conv_same.build(tf.TensorShape(images.shape))

conv_same.set_weights(conv_valid.get_weights())

same_output = conv_same(images.astype(np.float32))

assert np.allclose(valid_output.numpy(), same_output.numpy())

# Pooling layer

## Max pooling

In [ ]:
max_pool=keras.layers.MaxPool2D(pool_size=2)

In [ ]:
cropped_images= np.array([crop(image) for image in images], dtype = np.float32)
output= max_pool(cropped_images)

In [ ]:
fig=plt.figure(figsize =(12,8))
gs = mpl.gridspec.GridSpec(nrows=1, ncols=2, width_ratios=[2,1])

ax1= fig.add_subplot(gs[0,0])
ax1.set_title('Input', fontsize =14)
ax1.imshow(cropped_images[0])
ax1.axis('off')
ax2= fig.add_subplot(gs[0,1])
ax2.set_title('Output', fontsize =14)
ax2.imshow(output[0])
ax2.axis('off')

## Depth-wise pooling

In [1]:
class DepthMaxPool(keras.layers.Layer):
    def __init__(self, pool_size, strdies=None, padding="VALID", **kwargs):
        super().__init__(**kwargs)
        if strides is None:
            strides = pool_size
        self.pool_size = pool_size
        self.strides= strides
        self.padding = padding
    def call(self, inputs):
        return tf.nn.max_pool(inputs,
                             ksize = (1,1,1, self.pool_size),
                             strides = (1, 1, 1, self.pool_size),
                             padding = self.padding)

NameError: name 'keras' is not defined

In [ ]:
depth_pool = DepthMaxPool(3)
with tf.device("/cpu:0"):
    depth_output = depth_pool(cropped_images)
depth_output.shape


In [ ]:
depth_pool = keras.layers.Lambda(lambda X: tf.nn.max_pool(
    X, ksize = (1,1,1,3), strides = (1,1,1,3), padding = 'VALID'))
with tf.device("/cpu:0"):
    depth_output = depth_pool(cropped_images)
depth_output.shape


In [ ]:
plt.figure(figsize= (12,6))
plt.subplot(1, 2,1)
plt.title("Input", fontsize =14)
plot_color_image(cropped_images[0])
plt.subplot(1,2,2)
plt.title("Output", fontsize =14)
plot_image(depth_output[0, ..., 0])
plt.axis('off')

## Average pooling

In [ ]:
avg_pool = keras.layers.AvgPool2D(pool_size=2)

In [ ]:
output_avg - avg_pool(cropped_images)

In [ ]:
fig = plt.figure(figsize=(12,8))
gs = mpl.gridspec.GridSpec(nrows=1, ncols=2, width_ratios=[2,1])

ax1= fig.add_subplot(gs[0,0])
ax1.set_title('Input', fontsize =14)
ax1.imshow(cropped_images[0])
ax1.axis('off')
ax2= fig.add_subplot(gs[0,1])
ax2.set_title('Output', fontsize =14)
ax2.imshow(output_avg[0])
ax2.axis('off')

## Global Average Pooling

In [ ]:
global_avg_pool = keras.layers.GlobalAvgPool2D()
global_avg_pool(cropped_images)

In [ ]:
output_global_avg2 = keras.layers.Lambda(lambda X: tf.reduce_mean(X, axis=[1,2]))
output_global_avg2(cropped_iamges)

# Tackling Fashion MNIST With a CNN

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_mean= X_train.mean(axis=0, keepdims= True)
X_std =X_train.std(axis=0, keepdims= True)+ 1e-7
X_train= (X_train-X_mean)/X_std
X_valid= (X_valid-X_mean)/X_std
X_test= (X_test-X_mean)/X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]


In [ ]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding='SAME')

model=keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7,input_shape=[28,28,1]),
    keras.layers.MaxPooling2D(pool_size =2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size =2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size =2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

In [ ]:
model.compile(loss= 'sparse_categorical_crossentropy', optimizer = 'nadam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test)
X_new= X_test[:10]
y_pred=model.predict(X_new)

## ResNet-34

In [ ]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, stride=1,
                       padding='SAME', use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, stride =1, activation ="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides= strides),
            keras.layers.BatchNormalization(),
            self.activation, 
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers=[]
        if strides >1 :
            self.skip_layers =[
                DefaultConv2D(filters, kernel_size =1, strides =strides),
                keras.layers.BatchNormalization()]
            
    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z=layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z=layer(skip_Z)
        return self.activation(Z+skip_Z)

In [ ]:
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=7, strides =2,
                        input_shape=[224,224,3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2,padding="SAME"))
prev_filters = 64
for filters in [64 ]*3 +[128] * 4 + [256]* 6+[512]*3:
    strides = 1 if fiters == prev_filters else 2
    model.add(ResidualUnit(filters, strides = strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation = 'softmax'))


In [ ]:
model.summary()

## Using a Pretrained Model

In [ ]:
model=keras.applications.resnet50.ResNet50(weights="imagenet")

In [ ]:
images_resized=tf.image.resize(imagese, [224,224])
plot_color_image(images_resized[0])

In [ ]:
images_resized=tf.image.resize_with_pad(imagese, 224,224, antialias=True)
plot_color_image(images_resized[0])

In [ ]:
images_resized=tf.image.resize_with_crop_or_pad(imagese, 224,224)
plot_color_image(images_resized[0])

In [ ]:
china_box=[0,0.03,1,0.68]
flower_box=[0.19,0.26,0.86,0.7]
images_resized=tf.images.crop_and_resize(images,[china_box,flower_box],[0,1],[224,224])
plot_color_image(images_resized[0])
plot_color_image(images_resized[1])


In [ ]:
inputs=keras.applications.resnet50.preprocess_input(images_resized*255)
Y_proba=model.predict(inputs)

In [ ]:
Y_proba.shape

In [ ]:
top_K=keras.applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)):
    print("Image #{}".format(image_index))
    for class_id, name, y_proba in top_K[image_index]:
        print(" {} - {:12s} {:.2f}%".format(class_id, name, y_proba*100))
    print()

## Pretrained Models for Transfer Learning

In [ ]:
import tensorflow_datasets as tfds

dataset, info =tfds.load("tf_flowers", as_supervised=True, with_info=True)

In [ ]:
info.splits

In [ ]:
info.splits["train"]

In [ ]:
class_names=info.features["label"].names
class_names

In [ ]:
n_classes =info.features["label"].num_classes

In [ ]:
dataset_size = info.splits["train"].num_examples

In [ ]:
test_set_raw, valid_set_raw, train_set_raw =tfds.load(
    "tf_flowers", 
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

In [ ]:
plt.figure(figsize=(12,10))
index = 0
for image, label in train_set_raw.take(9):
    index+=1
    plt.subplot(3, 3, index)
    plt.imshow(image)
    plt.title("Class: {}".format(class_names[label]))
    plt.axis('off')